In [1]:
import os
os.chdir("..")
os.chdir("src")
from data_cleaning import DataCleaning
os.chdir("..")
from bertopic import BERTopic
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from bertopic.representation import KeyBERTInspired
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from transformers import pipeline
import torch
from datasets import Dataset
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer


2024-04-14 17:04:28.041340: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-14 17:04:28.042416: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-14 17:04:28.106113: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-14 17:04:28.367006: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 17:04:29.127082: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Sobre as métricas de avaliação
https://github.com/dice-group/Palmetto/issues/13

In [2]:
data_cleaning_pipeline = DataCleaning()
df = data_cleaning_pipeline.run()

data = Dataset.from_pandas(df)
docs = data["DS_OBJETO"]

2024-04-14 17:04:33 - INFO - Data loaded!
2024-04-14 17:04:33 - INFO - Null values cleaned!
2024-04-14 17:04:33 - INFO - Data types asserted!
2024-04-14 17:04:33 - INFO - Full data cleaned!


In [3]:
docs

['Contratação de empresa especializada em construção civil para, mediante execução indireta, em regime de empreitada por preço global, realizar a obra asfáltica  de recapeamento e pavimentação asfáltica na Linha Arroio Augusta , Interior, com área total de intervenção de 2.210m² (dois mil duzentos e dez metros quadrados), no Município de Roca Sales.',
 'Aquisição de gêneros alimentícios destinados para merenda escolar, nas escolas municipais de Educação Infantil Criança Feliz e de Ensino Fundamental Construindo o Saber, referente ao ano letivo de 2023.',
 'FORNECIMENTO DE HARDWARE E SOFTWARE DE LICENCIAMENTO DE FIREWALL, COMPATÍVEL COM O EQUIPAMENTO MODELO SONICWALL TZ670 PARA USO NO SETOR DE INFORMÁTICA, COM A FINALIDADE DE MONITORAMENTO, REGRAMENTO DE ACESSOS, CONTROLES DE BANDA, ENFIM, TUDO QUE SE REFERE À REDE DE COMPUTADORES DO MUNICÍPIO E DEMAIS MATERIAIS',
 'Aquisição de gêneros alimentícios perecíveis e não perecíveis para o atendimento ao Programa Nacional de Alimentação Escol

In [5]:
stop_words = []

with open("/home/brunodifranco/mestrado/mestrado-ufrgs-cmp617-tce/src/utils/stop_words.txt", "r") as file:
    for row in file:
        stop_words.append(row.strip())

vectorizer_model = CountVectorizer(
    stop_words=stop_words,
    strip_accents="unicode"
)

In [6]:
embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"

sentence_model = SentenceTransformer(model_name_or_path=embedding_model, device=device)
embeddings = sentence_model.encode(docs, show_progress_bar=True)

Batches:   0%|          | 0/4057 [00:00<?, ?it/s]

In [7]:
from cuml.preprocessing import normalize
embeddings = normalize(embeddings)

In [10]:
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_cluster_size=500, min_samples=10, gen_min_span_tree=True, prediction_data=True)

In [16]:
# min_topic_size = 500


# umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0, metric="cosine")
# hdbscan_model = HDBSCAN(
#     min_cluster_size=min_topic_size,
#     gen_min_span_tree=True,
#     prediction_data=True,
# )

# Pass the above models to be used in BERTopic
topic_model = BERTopic(
    nr_topics="auto",
    top_n_words=5,
    embedding_model=embedding_model,
    language="brazilian portuguese",
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    representation_model=KeyBERTInspired(),
)

In [17]:
topics, probs = topic_model.fit_transform(docs, embeddings)

In [20]:
topic_model.get_topics()

{0: [('destinacao', 0.62335056),
  ('conservacao', 0.5984051),
  ('construcao', 0.5922392),
  ('empreitada', 0.5921137),
  ('servico', 0.5731201)],
 1: [('hospitalares', 0.7581874),
  ('hospitalar', 0.74740803),
  ('hospital', 0.7159331),
  ('ambulatoriais', 0.66046727),
  ('medicos', 0.65740275)],
 2: [('construcao', 0.58435833),
  ('infraestrutura', 0.5800067),
  ('praca', 0.54473317),
  ('concreto', 0.52755153),
  ('recursos', 0.45180136)],
 3: [('alimenticios', 0.6818936),
  ('alimentar', 0.6663583),
  ('alimentos', 0.65172994),
  ('alimentares', 0.6341733),
  ('dietas', 0.52151966)],
 4: [('0km', 0.7868212),
  ('km', 0.65773183),
  ('transporte', 0.5765436),
  ('motorista', 0.54102445),
  ('transito', 0.5275573)],
 5: [('escolares', 0.69268394),
  ('escolas', 0.68687797),
  ('escolar', 0.683315),
  ('educacionais', 0.6445811),
  ('educacional', 0.6435565)],
 6: [('agricultura', 0.5849966),
  ('alimenticios', 0.5602207),
  ('alimentos', 0.5281632),
  ('agricultores', 0.5277628),
  

In [19]:
# topic_model = BERTopic(
#     nr_topics="auto",
#     embedding_model="paraphrase-multilingual-MiniLM-L12-v2", 
#     language="brazilian portuguese",
#     min_topic_size=500,
#     representation_model=KeyBERTInspired(),
#     vectorizer_model=vectorizer_model

# )
# topics, probs = topic_model.fit_transform(docs)

# IMPORTANTE

- Testar outros modelos de embeddings, de preferencia alguns mais pesados

In [19]:
cleaned_docs = topic_model._preprocess_text(docs)
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]

dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topics = topic_model.get_topics()
topics.pop(-1, None)
topic_words = [
[word for word, _ in topic_model.get_topic(topic) if word != ""] for topic in topics
]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
        for topic in range(len(set(topics))-1)][:10]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                          texts=tokens, 
                          corpus=corpus,
                          dictionary=dictionary, 
                          coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

0.6717193008241118

In [21]:
topics_red = topic_model.reduce_topics(docs, nr_topics=7)

In [22]:
topics_red.get_topics()

{-1: [('familia', 0.5450449),
  ('empreitada', 0.49136066),
  ('predio', 0.48912513),
  ('acessorios', 0.46785116),
  ('01', 0.46406507)],
 0: [('educacao', 0.5716433),
  ('construcao', 0.5696934),
  ('recursos', 0.5399675),
  ('abastecimento', 0.5359013),
  ('ensino', 0.52651286)],
 1: [('pneus', 0.66772383),
  ('maquinas', 0.5594555),
  ('automotor', 0.50165486),
  ('maquina', 0.4767235),
  ('motor', 0.47344762)],
 2: [('alimenticios', 0.66734236),
  ('alimentos', 0.63560534),
  ('agricultura', 0.6165323),
  ('agricultores', 0.5391453),
  ('hortifrutigranjeiros', 0.521094)],
 3: [('clinicas', 0.68132776),
  ('medicos', 0.6788057),
  ('hospitais', 0.6720931),
  ('clinica', 0.6619951),
  ('clinico', 0.6302252)],
 4: [('lampadas', 0.6660093),
  ('lampada', 0.61288285),
  ('iluminacao', 0.59820795),
  ('refletores', 0.58481073),
  ('refletor', 0.58268005)],
 5: [('veterinaria', 0.69685334),
  ('veterinarios', 0.6938695),
  ('veterinario', 0.6912789),
  ('caninas', 0.5350579),
  ('caninos

In [23]:
cleaned_docs = topics_red._preprocess_text(docs)
vectorizer = topics_red.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topics = topics_red.get_topics()
topics.pop(-1, None)
topic_words = [
[word for word, _ in topics_red.get_topic(topic) if word != ""] for topic in topics
]
topic_words = [[words for words, _ in topics_red.get_topic(topic)] 
        for topic in range(len(set(topics))-1)][:10]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                          texts=tokens, 
                          corpus=corpus,
                          dictionary=dictionary, 
                          coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

0.623689483185865

In [27]:
import optuna
def objective(trial):
    # Parâmetro a ser otimizado: número de tópicos
    nr_topics = trial.suggest_int('nr_topics', 4, 20)

    # Redução de tópicos
    topics_red = topic_model.reduce_topics(docs, nr_topics=nr_topics)
    cleaned_docs = topics_red._preprocess_text(docs)
    vectorizer = topics_red.vectorizer_model
    analyzer = vectorizer.build_analyzer()
    tokens = [analyzer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]
    topics = topics_red.get_topics()
    topics.pop(-1, None)
    topic_words = [
        [word for word, _ in topics_red.get_topic(topic) if word != ""] for topic in topics
    ]
    topic_words = [[words for words, _ in topics_red.get_topic(topic)] 
                    for topic in range(len(set(topics))-1)][:10]

    # Avaliação da Coerência
    coherence_model = CoherenceModel(topics=topic_words, 
                                     texts=tokens, 
                                     corpus=corpus,
                                     dictionary=dictionary, 
                                     coherence='c_v')
    coherence = coherence_model.get_coherence()

    return coherence

# Otimização com Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)  # Defina o número desejado de tentativas

# Obtendo o melhor número de tópicos e a melhor coerência
best_nr_topics = study.best_params['nr_topics']
best_coherence = study.best_value

print("Melhor número de tópicos:", best_nr_topics)
print("Melhor coerência encontrada:", best_coherence)

[I 2024-04-13 23:07:24,887] A new study created in memory with name: no-name-914aa124-5a61-4743-a62b-6c84116dca52
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if poss

Melhor número de tópicos: 18
Melhor coerência encontrada: 0.4901343860001878
